# Main Code used for Decoding
### Documentation created by Ana Vedoveli in 24 Feb 2017

This code is the first code used for decoding. It is divided in four principal parts (whole-trial decoding of conditions, whole trial decoding of directions, time-decoding of conditions and time-decoding of directions), in which the classifier used is always a linear support vector machine and I would try to replicate the protocol in JR King's papers. It also contains *extra* parts (not necessarily extra, but for the lack of a better name I called like this), in which I tried new classifiers or techniques to address the same data. 

Over this documentation, I'll provide a tutorial and try to explain what each part of the code is doing. Hopefully this will make the generalization and improvement of this code easier. I will also highlight parts in which I used other people's example, with links for them.


Is important to know that in this tutorial we are using the **struct_cor** data. The struct_cor contains the data that was filtered and baseline corrected. The same steps can be used when decoding the **struct_inter**, the data that was filtered, baseline corrected and had the missing channels interpolated. 

***

### Importing toolboxes and creating it's own functions.

In python, we import the functions of the different toolboxes we are using throughout the whole code. This is a similar idea to other languages, like R! and LaTeX. My personal preference is to import all the functions in the beginning of the code. That is what this next part is doing:

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 24 09:11:13 2018

@author: Ana

CI: time decoding of directions and conditions
"""

## Importing packages and defining functions.

import os
import matplotlib.pyplot as plt
from scipy.io import (loadmat, savemat)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
import numpy as np
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.preprocessing import LabelEncoder
from mne.decoding import LinearModel
from mne.decoding import get_coef
import mne
from sklearn.preprocessing import LabelBinarizer
from sklearn import metrics
from scipy import interp




### Defining your own particular functions

Similarly to R! and Matlab, you can create your own functions in Python. Differently from MatLab, however, you functions can be defined throught your code (another option is to have another .py file only with all your functions together). Python can have different consoles (like a command line), which means that if you open a new python console/window, you have to define the functions again. My personal preference also was to define all my own functions in the begining of the code. But this, as always, is a personal preference. 

Here I am defining two functions: the function **scorer**, taken from the JR King tutorials, and the function **load_cor**. The function **scorer** is responsible to scoring an AUC value over the probabilistic output of the SVM. The scoring of an AUC value *on the probabilistic output* is something that was not naturally implemented in the scikit-learn package: Usually, on scikit learn, if you ask your SVM function to give as an output a AUC value, this AUC would be automatically computed on the categorical output of the SVM, and not over the probabilistic output. Therefore, we needed to implement our own scorer and implement it on the cross-validation step.

The function **load_cor** is probably the most important function we have so far. My biggest problem was: how could I export a structure in the format of EPOCHS to the MNE? [There is a good page on the Fieldtrip website explaining how to export data from Fieldtrip to MNE](http://www.fieldtriptoolbox.org/development/integrate_with_mne), however this is still in development: you can easily export data from fieldtrip to MNE if they are on the format of RAW or EVOKED, but not EPOCHS (the format of the cleaned data). For a long time in my lb rotation, it was really hard for me to import data from fieldtrip to Python in a way I could keep all the important features I wanted (the data, the time window, the fs, etc) and to be adapted to the MNE-workflow. After trying many codes that did not succeed, I finally found someone who had the same problem as mine. I could adapt their codes with what I had tried so far. This function is, in my opinion, really important -- so if you haven't read the documentation about this function, you should do it before keep reading this one. Importantly, this code is supposed to run on Windows. A small adaptation of function load_cor was created for mac users.

In [2]:
def scorer(y_true, y_pred):
    # Probabilistic estimates are reported for each class. In our case 
    # `y_pred` shape is (n_trials, 2), where `y[:, 0] = 1 - y[:, 1]`.
    return roc_auc_score(y_true, y_pred[:, 1])    

## This code is meant to work on WINDOWS. Giulia has a version that works on MAC. 
def load_cor(xDir, var_name='struct_cor'):
    import mne
    from mne import create_info
    from mne.epochs import EpochsArray
    import scipy.io as sio
    import numpy as np
    # load Matlab/Fieldtrip data
    mat = sio.loadmat(xDir, squeeze_me=True, struct_as_record=False)
    ft_data = mat[var_name]
    event = ft_data.trialinfo[:, 1]

    # convert to mne
    n_trial, n_chans, n_time = ft_data.trial.shape
    data = np.zeros((n_trial, n_chans, n_time))
    data = ft_data.trial

    sfreq = 200
    time = ft_data.time

    
    coi = range(n_chans)
    data = data[:, coi, :]
    chan_names = [l.encode('ascii') for l in ft_data.label[coi]]
    chan_types = ft_data.label[coi]
    chan_types[:] = 'eeg'
    info = create_info(chan_names, sfreq, chan_types)
    events = np.array([np.arange(n_trial), np.zeros(n_trial), event], int).T
    epochs = EpochsArray(data, info, events=events,
                         tmin=np.min(time), verbose=False)
    montage = mne.channels.read_montage('GSN-HydroCel-257')
    epochs.set_montage(montage)
    return epochs, ft_data.trialinfo


---

## Starting the decoding script properly.

This is the start of the code per se. An important thing to note here is that I would personally prefer to have each step of the decoding pipeline to run over all the subjects I had. So, first, I define a *main directory*, in which I have all subject data. In the main directory, each subject has its own folder (named by the name of the subject), and inside of this folder I have the data I will use over the . 

After this, I use the listdir function (from the package os) to read the name of all the folders I have inside of my main directory (in other words, the name of the subjects), and I sort them alphabetically. I also try to select only the subjects cleaned with the third method (v03).


In [4]:


## Changing the main directory of your files.
mainDir = 'C:\\Users\\Ana\\Desktop\\CI\\Python\\Subjects\\' # Change this to your own path

# Start: making list of subject names.
subject = os.listdir(mainDir)
subject = np.sort(subject) # variable containing a list of subjects in our folder.

## Selecting subjects only for the version 03 of cleaning. Skip this step if want
# to go through all subjects.

#temp =[]
#for name in subject:
#    if name[-3:] == 'v03':
#        temp.append(name)
#subject = np.asarray(temp)

###### Now that we have our own list of subjects, we will start the decoding properly.
---

###  First part of analysis: decoding whole-trial of conditions

Now we will start with the easiest question to answer: if we look at the whole trial (n_channels x n_time points), can we distinguish the trials in which the condition was same and when it was different? This decoding step yields a single performance value for each subject. Remember that here we will loop through all the subjects we have on our "subject" variable. The following code is commented to help following the logic:

In [ ]:
mean_fpr = np.linspace(0, 1, 100)
# notice that in python indentation is important. Here we start a for loop, 
# over every name in the subject list.
for name in subject:
    
	# In python, you have to pre-name all variables that you are going to use
	# beforehand. That's what we are doing here.
    score_conditions = [] # in this variable, we have the cross-validated 
                          # AUC value (average across folds).
        
    score_tprs =[] # in this variable, we have the cross validated
                   # True positive rate (average across folds).
    
    
    filePath = mainDir + name # main path for the subject
    #loading labels for conditions - we have a variable trl_conditions
    # containing the labels (same: -1, different: 1) for each trial of 
    # that subject. 
    yDir = filePath + '\\trl_conditions.mat' # directory of the file
    Y = loadmat(yDir) # loading the file with function loadmat
    conditions = Y['trl_conditions'] # loadmat gives us many things we don't need 
                                     # (including the header of the 
                                     # file. we retrieve only the data we want.
        
    Y = conditions.transpose().ravel() # we rewrite variable Y with only the data 
                                       # we want (that is stored in conditions). 
                                       # We transpose the data and use the ravel method to get 
                                       # an one dimensional vector. This happens because the SVM
                                       # function only accepts one dimmensional vectors as labels
    
    # In python, all these steps could have been done in only one line. 
    # However, I kept it this way so we can  follow the logic easily. 
    # I also change all the value from -1 to 0. As I understood the idea of 
    # scoring an AUC# over the probabilistic output, it would make more sense that the
    # labels were 0 and 1 instead of 1 and -1. Another way of solving this for good is 
    # to change the value from -1 to 0 in the matlab function "CI_defineCondition".
        
    Y[Y==-1] = 0
    
    # Now we are going to load the EEG data as epoch object
    xDir = mainDir + name + '\\struct_cor.mat'
    epochs, _ = load_cor(xDir, var_name='struct_cor')
    
    # OBS: the epochs object in python is known as "class". A class is a type of object 
    # in python, that has functions in-built. This means, nothing more, that a class is
    # a special variable that has its own functions  (in python we call these functions
    # methods). Note that "epochs" won't show in the variable explorer in spyder. To check 
    # if you loaded your data properly, type epochs in command line and see what you get.
    
    #Retrieving data as matrix
    data = epochs.get_data() # here is one example of a method (special funtion) - 
                             # get_data() gets the EEG data inside the epoch class.
    
    X = np.reshape(data, [data.shape[0], data.shape[1]*data.shape[2]]) # what we are you doing here is 
                                                                       # putting all the channels 
                                                                       # and time points as features for the classifier.
    
    ## classification problem "conditions".
    
    # Now we will define all the objects that are import for us. We will define a classifier
    # clf, that will perform thefollowing steps: it will scale the data, perform a dimensionality 
    # reduction (selectkbest) keeping the best 20 features  and, in the end, it will fit a SVM 
    # (SVC - Support vector classifier). 
    clf = Pipeline([('scaling', StandardScaler()), ('SelectKBest', SelectKBest(f_classif, k=20)), ('svc',  SVC(class_weight='balanced', probability=True, kernel='linear'))])
    
    # Now we are also defining an object for the stratified cross-validation 'cv'. 
    # Object cv will split the data in 5 stratified
    # folds, shuffling it to avoid any carry-over effect
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    
    # to fit your classifier for the data, you use the method 'fit':
    clf.fit(X, Y)

    
    ## However, we need to cross-validate our classifier. We will do this in a loop and do 
    # the cross validation step-by-step. Note that in python this wouldn't be necessary. 
    # You could perform your cross-validation in one line (i.e, you could use 
    # the cross_val_score function from scikit learn!). That is also possible. 
    # You could do something like: 
    # auc = cross_val_score(clf, X, Y, cv=cv, scoring=scorer)
    # both ways are fine. I just thought that, personally, it was better for me to learn python 
    # by doing things step by step, even if I knew there were more simple ways of doing it.
    # Why not trying your own way and see what fits you best? :D
    # Well, let's go back to our cross-validation:
    
    
    tprs = [] # this variable will take the all the tprs per fold. 
              # This should give you a variable with n_folds list.
        
    score = [] # this variable will give you all the scores per fold. 
               # This will be a vector of size n_folds.
    
    for train, test in cv.split(X=X, y=Y): # starting our cross-validation. In this for loop, we take 
                                           # every train and test group (note that python allows you 
                                           # to have for loops with more than one variable) in the
                                           # cv.split. What the method split is doing is just splitting
                                           #  our data in the parameters we specified on our cv object. 
                                           # train and test gives us a list of index.
                                           # we use this list to index our X and Y.
            # Fit on train set 
            clf.fit(X[train, :], Y[train])
            # Predict on test set
            y_pred = clf.predict_proba(X[test, :])
            
            # use the roc_curve function to get the fpr and tpr of the fold
            fold_fpr, fold_tpr, _ = metrics.roc_curve(Y[test], y_pred[:, 1])
            # we score the performance of our classifier for this fold
            fold_auc = scorer(Y[test], y_pred)
            # we then append it in the score and tprs variables. 
            # To append just means to add a list to a variable 
            # (to append, really)
            score.append(fold)
            tprs.append(interp(mean_fpr, fold_fpr, fold_tpr))
            tprs[-1][0] = 0.0
    # Now that we have calculated all the folds we want to know the average across folds. 
    # We take the average both for the tprs and the score.         
    score_tprs.append(np.mean(tprs, 0))
    score_conditions.append(np.mean(score))
    
    # Now we save the cross-validated score and tprs.
    np.save(filePath + '\\tprs', score_tprs) 
    np.save(filePath + '\\score_conditions', score_conditions) 
    print('The scores for subject ' + name + 'is ' + str(score_conditions))

***OBS***: I am not sure how the function roc_score works. I mainly looked at the documentation online for creating it. I asked more information for the python community and I am waiting for an answer.

---

###  Second part of analysis: decoding whole-trial of directions

Now we will use an one-versus-rest strategy to decode directions taking as features the whole trial (n_chan x n_times). The next part of the code is quite similar to the previous. I will try to focus my comments on the things that are different from the previous part.

In [ ]:

for name in subject:    
	filePath = mainDir + name
    
    #loading data as epoch object
    xDir = filePath + '\\struct_cor.mat'
    epochs, _ = load_cor(xDir, var_name='struct_cor')
    
    #Retrieving data as matrix
    data = epochs.get_data()
    X = np.reshape(data, [data.shape[0], data.shape[1]*data.shape[2]])

    # Differently from the last part, the labels for the directions are 
    # already inside of the epochs object! So we don't need to load anything
    # as labels. We just need to retrieve them using the method '.event'. 
    # The third column of events is the column that contains the information
    # we need (if you are curious why, have a look on how MNE define events). 
    # As it is the third column, the index is number 2 (remember, python 
    # starts indexation from 0)
    directions = epochs.events[:, 2]
    
    # As we are using an OVR strategy, we need to transform the 
    # labels '2, 3, 4, 5, 6' into 5 matrix of ones and zeros. The
    # object LabelBinarizer will help us with this job. We should 
    # get in the end a matrix with the number of observations 
    # (trials) in the first dimension and the number of direction 
    # in the second one
    
    lb = LabelBinarizer()
    Y = lb.fit_transform(directions) # the method .fit_transform creates 
                                     # the desired matrix for us.
    
    
    ## Now we will start with the loop over DIRECTIONS. 
    n_classes = 5                  
    directions_scores = []  # this variable will contain in the end 
                            # the cross-validated AUC 
                            # values for each direction for the given subject 
            
    for directions in range(n_classes):
        y=Y[:, directions] # here we get the set of labels we want. For instance, 
                           # if we want the labels for the first direction
                           # the variable directions will be 0 and we will get 
                           # the OVR labels for the first direction, and so on.
                
                
        clf = Pipeline([('scaling', StandardScaler()), ('SelectKBest', SelectKBest(f_classif, k=20)), ('svc',  SVC(class_weight='balanced', probability=True, kernel='linear'))])
        cv = StratifiedKFold(n_splits=5, shuffle=True)

        score = [] # this contains a collection of AUC values for each fold
        for train, test in cv.split(X=X, y=y):
                # Fit on train set
                clf.fit(X[train, :], y[train])
                # Predict on test set
                y_pred = clf.predict_proba(X[test, :])
        
                auc_fold = scorer(y[test], y_pred)
                score.append(auc_fold)
        directions_scores.append(np.mean(score)) # here we append the cross-validated 
                                                 # score for every direction.
                                                 # be careful with indentation!!!
    # once everything is done, save the data.        
    np.save(filePath + '\\score_directions', directions_scores) 
    print('Whole trial decoding of directions for subject ' + name[0:8] + 'ready')     

### Third part of analysis: time decoding of "conditions".

After completing the whole trial decoding, it is time to perform the time-decoding for conditions. This means that now we will train one classifier per each time point. This will yield an 1 x n_timepoints vector containing the AUC values for each time point. 

In this step, we will start retrieving the spatial patterns and filters for each time point. This is the data used to plot the topography I showed in my presentation. For getting the spatial filter and patterns, we use the function LinearModel from MNE,  that was built based on the famous paper. An important observation about how I use this function is mentioned throughout the code. 


In [ ]:
for name in subject:
    
    filters = [] # this variable will contain the filters for the given subject
    scores = [] # this variable will contain the cross-validated AUC 
                # over time for the given subject
    patterns =[] # this variable will contain the patterns for the given subject
       
    
    filePath = mainDir + name 
    
    # We start the code similarly to part one. 
    # However, I added this check to verify if I
    # had already decoded this subject
    # before, to save some time. 
    
    if  os.path.isfile(filePath + '\\subject_conditions_time.npy') == False:
        
        #loading labels for conditions
        yDir = filePath + '\\trl_conditions.mat'
        Y = loadmat(yDir)
        conditions = Y['trl_conditions']
        Y = conditions.transpose().ravel()
        Y[Y==-1] = 0
            
        
        #loading data as epoch object
        xDir = mainDir + name + '\\struct_cor.mat'
        epochs, _ = load_cor(xDir, var_name='struct_cor')
        data = epochs.get_data() # data has size: n_trials x n_channels x n_timepoints
        
        ## Now we are adding one more loop: a loop over time. This is the part that implements the over time decoding.    
        n_time = np.size(data, 2)
        for point in np.arange(n_time): 
           
            # Here we get the topography from the data in time point 'point'
            X = data[:, :, point]
            
            # Here I create two different types of classifier objects: clf and model. 
            # Clf will perform a dimensionality reduction, selecting the  most relevant 
            # channels when doing the decoding. Model, on the other hand, will take 
            # all the features (all the channels) to retrieve more reliable filters
            # and patterns. I did that because keeping all the features when
            # decoding made everything significantly slower. In the JR King papers, 
            # they do NOT perform a dimensionality reduction during the time decoding.
            # But this made every subject take about ~5 days in my computer. 
            # his decision was just I could optimize my time. With a cluster, you can 
            # play around with the  dimensionality reduction (how, how many features, 
            # reducing it or keeping all of them). 
            # This is definitely a point that can be improved.
            
            clf = Pipeline([('scaling', StandardScaler()), ('SelectKBest', SelectKBest(f_classif, k=20)), ('svc',  SVC(class_weight='balanced', probability=True, kernel='linear'))])       
            model = Pipeline([('scaling', StandardScaler()), ('svc',  LinearModel(SVC(class_weight='balanced', kernel='linear')))])
            cv = StratifiedKFold(n_splits=5, shuffle=True)
            
            ## Retrieving the patterns and filters for that time point.
            model.fit(X, Y)            
            pat = get_coef(model,  'patterns_',  inverse_transform=True) # pat contains the patterns 
                                                                         # values for time point
                
            filt = get_coef(model,  'filters_',  inverse_transform=True) # filt contains the filters 
                                                                         # values for time point
            
            # we append both pat and filt for this timepoint in patterns
            # and filters. Patterns and filters will contain many
            # list (n_timepoints lists) containing the patterns 
            # and filters for the whole epoch.
            patterns.append(pat)        
            filters.append(filt)
            
            # Now we perform the decoding for the timepoint. 
            # This is similar to what we saw before
            score = []
            for train, test in cv.split(X=X, y=Y):           
                
                # Fit on train set
                clf.fit(X[train, :], Y[train])
                # Predict on test set
                y_pred = clf.predict_proba(X[test, :])
        
                fold_auc = scorer(Y[test], y_pred)
                score.append(fold_auc)
            
            scores.append(np.mean(score))
                    
            print('Subject' + name[0:8] + ', time point: ' + str(point)) 
            
        np.save(filePath + '\\subject_conditions_time', scores) 
        np.save(filePath + '\\subject_conditions_patterns', patterns) 
        np.save(filePath + '\\subject_conditions_filters', filters) 
        print('Time decoding for subject ' + name[0:8] + 'are ready')  
    else:
        print('Time decoding already ready')

### Fourth part of analysis: time decoding of "directions".

Now we will use the same OVR strategy over time. I would like you to keep in mind how many classifiers we will train now: every epoch has 611 time points (so 611 classifiers). With the 5-fold cross validation, we will train 5 classifiers per time point - so 5 * 611 = 3055 classifiers. But as we will use a one-versus-rest approach, this means that we will train 3055 classifiers for every 5 directions, so again 3055 * 5 = 15275. So for each subject, we train in total at least 15275 classifiers. If you multiply this number by the number of subjects, *this number will be even higher*. So it is not a surprise that this step is extremly slow, taking many days to finish decoding one subject -- even if you are using the cluster. Here I decided to import the package time to have an idea of how many seconds it takes to decoding on time point.

Keeping this in mind, the code is structure in many loops, that are embedded in one another in this order: the first loop is across subjects, the second loop is across directions, the third loop is across time points, and the fourth loop is the cross validation. By now, the general structure of the workflow should be familiar for you, so comments will be restricted to the new features of this step.

In [ ]:
import time 

for name in subject: 
        
    print('SVM for subject ' + name)
    filePath = mainDir + name
    
    #loading data as epoch object
    print('Loading data')
    xDir = filePath + '\\struct_cor.mat'
    epochs, _ = load_cor(xDir, var_name='struct_cor')
    
    #Retrieving data as matrix
    data = epochs.get_data()
    
    #Retrieving labels for directions - remember, this is a similar step to what was done before (part 2)
    directions = epochs.events[:, 2]
    
    lb = LabelBinarizer()
    Y = lb.fit_transform(directions)
    
    n_classes= np.size(Y, 2)
    directions_time_scores =[] # will contain collections of n_directions 
                               # lists containing the scores over time.
        
    directions_patterns=[]     # will  collections of n_directions lists 
                               #containing the patterns over time.
        
    directions_filters=[]      # will  collections of n_directions 
                               # lists containing the filters over time.
    for directions in range(n_classes):
        print('directions ' + str(directions))
        y=Y[:, directions]
        
        print('defining models')
        clf = Pipeline([('scaling', StandardScaler()), ('SelectKBest', SelectKBest(f_classif, k=20)), ('svc',  SVC(class_weight='balanced', probability=True, kernel='linear'))])             
        model =  Pipeline([('scaling', StandardScaler()), ('svc',  LinearModel(SVC(class_weight='balanced', probability=True, kernel='linear')))])             
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        print('models defined')
        
    
        n_time = np.size(data, 2)             
        scores = [] # vector of scores over time for a given direction
        patterns = []  # group of lists of patterns over time for a given direction
        filters = [] # group of lists of filters over time for a given direction
        for point in np.arange(n_time):
            t0 = time.time() # this is used by package time to count time. similar to tic toc in matlab
            print('computing SVMs for time point ' + str(point))
            X = data[:, :, point]
            
            print('fitting patterns')
            model.fit(X, y)            
            pat = get_coef(model,  'patterns_',  inverse_transform=True)
            filt = get_coef(model,  'filters_',  inverse_transform=True)
            print('patterns ready')
            patterns.append(pat)
            filters.append(filt) 
            
            score = [] # group of scores for each fold for a given time point
            n = 0
            for train, test in cv.split(X=X, y=y):
                n = n+1
                print('cv fold number ' + str(n))
                # Fit on train set
                clf.fit(X[train, :], y[train])
                # Predict on test set
                y_pred = clf.predict_proba(X[test, :])
                fold_auc = scorer(y[test], y_pred)
                score.append(fold_auc)
            t1 = time.time()
            print('taking the mean of the fold')
            scores.append(np.mean(score)) # cross validated AUC for time point
            
            total = (t1-t0) # this is for counting the time
            print('Subject' + name[0:8] + ', time point: ' + str(point) + 'total time = ', str(total))
        directions_time_scores.append(scores) #  collections of n_directions lists 
                                              # containing the scores over time.
            
        directions_patterns.append(patterns)  #  collections of n_directions lists 
                                              # containing the patterns over time.
            
        directions_filters.append(filters)    #  collections of n_directions lists 
                                              # containing the filters over time.   
    
    np.save(filePath + '\\subject_directions_time', directions_time_scores) 
    np.save(filePath + '\\subject_directions_patterns', directions_patterns)
    np.save(filePath + '\\subject_directions_filters', directions_filters)  
    print('Time decoding of direction for subject ' + name[0:8] + 'are ready')  

---

# The Extra part

**Why an "extra" part?**

During my rotation, I've tried many things - some of them would work, some of them not. I decided to say that everything that used an SVM should be the "main part" -- because I thought that I would be able to compare with all the other papers I had read about decoding of EEG data (in which they use mainly SVMs). However, at some point, I decided that I could also try other *classifiers*. I call this part of the script in which I try different models as "extra". 

I first tried to use [support vector regression](http://scikit-learn.org/stable/modules/kernel_ridge.html) to do an ordinal regression on the data. As every loudspeaker is ordered, this could be a way of reducing the number of classifiers I had to train in the direction problem and thus give a quicker answer I could show in the presentation. I inspired in the method used by JR King for ordinal regression. That is what I implement in the following steps.

### Whole trial decoding of directions using SVRs.

Repeating second part of analysis (decoding directions) using linear SVR. The result will be a single value for each subject.  

In this part, we are setting a new scorer: The measure performance here is the non-parametric rho correlation value. We also import the LinearSVR function

In [5]:
from sklearn.svm import LinearSVR

# Scorer: non parametric R²
def scorer(y_true, y_pred):
    from scipy.stats import spearmanr
    rho, p = spearmanr(y_true, y_pred)
    return rho


for name in subject:
    directions_svr_scores = []
    print('decoding subject ' + name)
    filePath = mainDir + name
   
    #loading data as epoch object
    xDir = filePath + '\\struct_cor.mat'
    print('loading epochs')
    epochs, _ = load_cor(xDir, var_name='struct_cor')
    
    #Retrieving data as matrix
    data = epochs.get_data()
    print('reshaping data')
    X = np.reshape(data, [data.shape[0], data.shape[1]*data.shape[2]])

    y = epochs.events[:, 2]  
    
    
    # in order to make results more interpretable, we remove the trials in which the 
    # last sound was different.
    yDir = mainDir + name + '\\trl_conditions.mat'
    Y = loadmat(yDir)
    conditions = Y['trl_conditions']
    Y = conditions.transpose().ravel()
    
    X = X[Y==-1, :]
    y = y[Y==-1]

    ## From JR King python notebook
    # Estimator
    print('defining estimator')
    clf = Pipeline([('scaling', StandardScaler()), ('anova', SelectKBest(f_classif, k=20)),  ('svr', LinearSVR())])
    cv = StratifiedKFold(n_splits=5)
    y_test =[]
    r =[]
    # Fit, predict, and score
    print('creating cross val')
    i = -1
    for train, test in cv.split(X=X, y=y):
        i = i + 1
        print('computing fold: ', i)
        print('fitting...')
        clf.fit(X[train, :], y[train])
        print('predicting...')
        y_pred = clf.predict(X[test, :])
        r.append(scorer(y[test], y_pred))  # score in [-1, 1], chance = 0.
        print('score:', r)  # should be > 0.
    
    directions_svr_scores.append(np.mean(r))
    print('saving...')
    np.save(filePath + '\\directions_svr_scores', directions_svr_scores) 
    print('Final score for subject' + name + 'is: '+  str(np.mean(r)))

decoding subject AB171109_20171109_111227v03HT
loading epochs
reshaping data
defining estimator
creating cross val
('computing fold: ', 0)
fitting...
predicting...
('score:', [-0.019599838302001012])
('computing fold: ', 1)
fitting...
predicting...
('score:', [-0.019599838302001012, 0.24205800302971248])
('computing fold: ', 2)
fitting...
predicting...
('score:', [-0.019599838302001012, 0.24205800302971248, 0.29595755836021526])
('computing fold: ', 3)
fitting...
predicting...
('score:', [-0.019599838302001012, 0.24205800302971248, 0.29595755836021526, 0.22245816472771146])
('computing fold: ', 4)
fitting...
predicting...
('score:', [-0.019599838302001012, 0.24205800302971248, 0.29595755836021526, 0.22245816472771146, 0.28811762303941485])
saving...
Final score for subjectAB171109_20171109_111227v03HTis: 0.205798302171
decoding subject AB171219_20171219_045837v02CI
loading epochs
reshaping data
defining estimator
creating cross val
('computing fold: ', 0)
fitting...


C:\Users\Ana\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


predicting...
('score:', [-0.74586985398289152])
('computing fold: ', 1)
fitting...
predicting...
('score:', [-0.74586985398289152, -0.09323373174786144])
('computing fold: ', 2)
fitting...
predicting...
('score:', [-0.74586985398289152, -0.09323373174786144, -0.29665278283410457])
('computing fold: ', 3)
fitting...
predicting...
('score:', [-0.74586985398289152, -0.09323373174786144, -0.29665278283410457, -0.67283849489430991])
('computing fold: ', 4)
fitting...
predicting...
('score:', [-0.74586985398289152, -0.09323373174786144, -0.29665278283410457, -0.67283849489430991, 0.87208159927238094])
saving...
Final score for subjectAB171219_20171219_045837v02CIis: -0.187302652837
decoding subject AB171219_20171219_045837v03CI
loading epochs
reshaping data
defining estimator
creating cross val
('computing fold: ', 0)
fitting...
predicting...
('score:', [0.12164566778234287])
('computing fold: ', 1)
fitting...
predicting...
('score:', [0.12164566778234287, 0.35538757783504582])
('computing 

### Extra: using SVR scores with a pearson regression to decode directions. 

This eans each subject is going to have one temporal decoding time. Assuming that there is information in the data about direction, the r value would peak in the time point information is present.

In [ ]:
for name in subject:
    directions_svr_time_scores =[]
    print('decoding subject ' + name)
    filePath = mainDir + name
   
    #loading data as epoch object
    xDir = filePath + '\\struct_cor.mat'
    print('loading epochs')
    epochs, _ = load_cor(xDir, var_name='struct_cor')
    
    #Retrieving data as matrix
    data = epochs.get_data()
    print('reshaping data')

    y = epochs.events[:, 2]  
    
    # in order to make results more interpretable, we remove the trials in which the 
    # last sound was different.
    print('removing ~different~ conditions')
    yDir = filePath + '\\trl_conditions.mat'
    Y = loadmat(yDir)
    conditions = Y['trl_conditions']
    Y = conditions.transpose().ravel()
    
    data = data[Y==-1, :, :]
    y = y[Y==-1]
        
    
    scores = []
    n_time = np.size(data, 2)
    i = -1
    for point in np.arange(n_time):
        i = i + 1
        print('time point ' + str(i))
        X = data[:, :, point]
        ## From JR King python notebook
        # Estimator
        print('defining estimator')
        clf = Pipeline([('scaling', StandardScaler()), ('anova', SelectKBest(f_classif, k=20)),  ('svr', LinearSVR())])
        cv = StratifiedKFold(n_splits=5)
        y_test =[]
        r =[]
        # Fit, predict, and score
        print('creating cross val')
        q = -1
        r = []
        for train, test in cv.split(X=X, y=y):
            q = q + 1
            print('computing fold: ', q)
            print('fitting...')
            clf.fit(X[train, :], y[train])
            print('predicting...')
            y_pred = clf.predict(X[test, :])
            r.append(scorer(y[test], y_pred))  # score in [-1, 1], chance = 0.
            print('score:', r)  # should be > 0.
        scores.append(np.mean(r)) 
    directions_svr_time_scores.append(scores)
    print('saving...')
    np.save(filePath + '\\directions_svr_time_scores', directions_svr_time_scores) 
    print('Final score for subject' + name + 'is: '+  str(np.mean(r)))  